In [11]:
import pandas as pd
import numpy as np

def calculate_mrr(df, k=None):
    """
    Calculate Mean Reciprocal Rank (MRR) from a DataFrame containing true and predicted values.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing the true and predicted values
    true_column : str
        Name of the column containing the true/relevant items
    pred_column : str
        Name of the column containing the predicted items (as lists)
    k : int, optional
        Calculate MRR@k. If None, use full length of predictions
    
    Returns:
    --------
    float
        Mean Reciprocal Rank score
    """
    def get_reciprocal_rank(true_item, pred_list):
        if k is not None:
            pred_list = pred_list[:k]
        
        # Find the position (1-based index) of true item in predictions
        try:
            rank = pred_list.index(true_item) + 1
            return 1.0 / rank
        except ValueError:
            return 0.0  # True item not found in predictions
    # Calculate reciprocal rank for each row
    reciprocal_ranks = df.apply(
        lambda row: get_reciprocal_rank(
            row['true_item'], 
            row['predictions']
        ), 
        axis=1
    )
    
    # Calculate mean of reciprocal ranks
    mrr = reciprocal_ranks.mean()
    
    return mrr

In [24]:
import psycopg2

pg_host = "localhost"
pg_database = "graph"
pg_user = "postgres"
pg_password = "postgres"

# Establish PostgreSQL connection
pg_conn = psycopg2.connect(
    host=pg_host,
    database=pg_database,
    user=pg_user,
    password=pg_password
)
pg_cursor = pg_conn.cursor()

In [40]:
pg_cursor.execute("""
 with ctx_result as (
 	select * from result_option_llm where "method" = 'EUCLIDEAN'
	order by result_option_llm.source_id, result_option_llm.similarity desc
), 
ctx_result_true as (
	select source_id, option_id from result_option_llm where "method" = 'EUCLIDEAN' and is_similar = true
)
select jsonb_agg(ctx_result.option_id) as result, COALESCE(ctx_result_true.option_id, '3447aa7e-edf5-4b59-93b8-afede2c74805') as option from ctx_result
left join ctx_result_true on ctx_result_true.source_id = ctx_result.source_id
group by ctx_result.source_id, ctx_result_true.option_id
""")
result_sentence = pg_cursor.fetchall()
df_sentence = pd.DataFrame(result_sentence, columns=['result', 'option'])

In [41]:
pg_cursor.execute("""
 with ctx_result as (
 	select * from result_option_graph where "method" = 'COSINE'
	order by result_option_graph.source_id, result_option_graph.similarity desc
), 
ctx_result_true as (
	select source_id, option_id from result_option_graph where "method" = 'COSINE' and is_similar = true
)
select jsonb_agg(ctx_result.option_id) as result, COALESCE(ctx_result_true.option_id, '40e68613-7ac2-4ffe-8a24-3a26ad453e91') as option from ctx_result
left join ctx_result_true on ctx_result_true.source_id = ctx_result.source_id
group by ctx_result.source_id, ctx_result_true.option_id
""")
result_graph = pg_cursor.fetchall()
df_graph = pd.DataFrame(result_graph, columns=['result', 'option'])

In [42]:
data = {
    'true_item': df_sentence['option'],
    'predictions': df_sentence['result']
}
data_graph = {
    'true_item': df_graph['option'],
    'predictions': df_graph['result']
}
df_combine_sentence = pd.DataFrame(data)
df_combine_graph = pd.DataFrame(data_graph)

In [43]:
mrr = calculate_mrr(df_combine_sentence)
print(f"sentence base LLM MRR: {mrr}")
mrr = calculate_mrr(df_combine_graph)
print(f"graph base  MRR: {mrr}")

sentence base LLM MRR: 0.4778225806451613
graph base  MRR: 0.6608208955223881
